In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **데이터 살펴보기**

In [ ]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [ ]:
#경로 설정
import os
os.chdir('/content/drive/My Drive/Dacon/소설작가분류')

In [ ]:
#파일 불러오기
train = pd.read_csv('open/train.csv', encoding = 'utf-8')
test = pd.read_csv('open/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [ ]:
#train 데이터 살펴보기
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [ ]:
#test 데이터 살펴보기
test

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [ ]:
#sample_submission
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


# **전처리**

In [ ]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [ ]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [ ]:
# 이상치 제거
train = train[train.text != "     "]
train = train.reset_index(drop=True)
train

In [ ]:
import nltk
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
# stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
#              "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
#              "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
#              "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
#              "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
#              "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
#              "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
#              "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
#              "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
#              "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
#              "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [ ]:
#from nltk.stem.porter import PorterStemmer
#stemmer = PorterStemmer()

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [ ]:
# 어간추출
def stemming(text):
    final_text = []
    for i in text.split():
      final_text.append(stemmer.stem(i))
    return " ".join(final_text)

In [ ]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)
train['text'] = train['text'].apply(alpha_num).apply(stemming)
test['text'] = test['text'].apply(alpha_num).apply(stemming)

In [ ]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [ ]:
X_train

array(['almost choke much much want say strang exclam came lip pole gaze fix bundl note hand look odin evid perplex',
       'sister ask suppos',
       'engag one day walk perus jane last letter dwell passag prove jane written spirit instead surpris mr odin saw look odin meet put away letter immedi forc smile said',
       ..., 'sincer wellwish friend sister luci odin', 'want lend money',
       'certain occur said yes like'], dtype='<U1310')

# **모델링**

In [ ]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [ ]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [ ]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 16)           320000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 125       
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/20
1372/1372 - 8s - loss: 1.5677 - accuracy: 0.2777 - val_loss: 1.5460 - val_accuracy: 0.3032
Epoch 2/20
1372/1372 - 8s - loss: 1.4245 - accuracy: 0.4000 - val_loss: 1.3229 - val_accuracy: 0.4467
Epoch 3/20
1372/1372 - 8s - loss: 1.2734 - accuracy: 0.4590 - val_loss: 1.2603 - val_accuracy: 0.4707
Epoch 4/20
1372/1372 - 7s - loss: 1.2070 - accuracy: 0.4879 - val_loss: 1.2002 - val_accuracy: 0.4962
Epoch 5/20
1372/1372 - 7s - loss: 1.1428 - accuracy: 0.5237 - val_loss: 1.1961 - val_accuracy: 0.4912
Epoch 6/20
1372/1372 - 7s - loss: 1.0808 - accuracy: 0.5604 - val_loss: 1.0978 - val_accuracy: 0.5482
Epoch 7/20
1372/1372 - 8s - loss: 1.0164 - accuracy: 0.5982 - val_loss: 1.0375 - val_accuracy: 0.5979
Epoch 8/20
1372/1372 - 7s - loss: 0.9533 - accuracy: 0.6320 - val_loss: 0.9991 - val_accuracy: 0.6150
Epoch 9/20
1372/1372 - 7s - loss: 0.9008 - accuracy: 0.6576 - val_loss: 0.9510 - val_accuracy: 0.6337
Epoch 10/20
1372/1372 - 7s - loss: 0.8539 - accuracy: 0.6776 - val_loss: 0.9268 - 

In [ ]:
# predict values
pred = model.predict_proba(test_padded)

In [ ]:
pred

array([[3.6947712e-01, 2.2731745e-01, 2.2239497e-01, 1.4138114e-02,
        1.6667233e-01],
       [6.5212108e-02, 1.5075187e-01, 1.7713133e-01, 1.1761373e-01,
        4.8929098e-01],
       [9.4337392e-01, 5.5225208e-02, 2.5458878e-04, 4.6741711e-10,
        1.1463386e-03],
       ...,
       [3.3429193e-03, 9.9343300e-01, 2.7754797e-06, 1.1985385e-06,
        3.2201458e-03],
       [5.0912215e-03, 9.8987365e-01, 7.6118445e-06, 1.2345179e-06,
        5.0263000e-03],
       [9.5339102e-01, 1.7103564e-05, 4.6577252e-02, 8.7241478e-06,
        5.8591163e-06]], dtype=float32)

In [ ]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,0.369477,2.273175e-01,2.223950e-01,1.413811e-02,1.666723e-01
1,1,0.065212,1.507519e-01,1.771313e-01,1.176137e-01,4.892910e-01
2,2,0.943374,5.522521e-02,2.545888e-04,4.674171e-10,1.146339e-03
3,3,0.000002,6.091814e-11,9.998645e-01,1.128897e-06,1.320531e-04
4,4,0.524426,1.030397e-01,7.228898e-02,2.899907e-01,1.025437e-02
...,...,...,...,...,...,...
19612,19612,0.000264,9.997361e-01,2.128808e-13,4.202517e-15,1.218690e-07
19613,19613,0.000011,2.064850e-04,1.393700e-03,1.606553e-10,9.983889e-01
19614,19614,0.003343,9.934330e-01,2.775480e-06,1.198538e-06,3.220146e-03
19615,19615,0.005091,9.898736e-01,7.611844e-06,1.234518e-06,5.026300e-03


In [ ]:
sample_submission.to_csv('submission/snow_submission.csv', index = False, encoding = 'utf-8')